In [3]:
! python training.py

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
INFO:__main__:Model trainable parameters:
 trainable params: 3145728 || all params: 1318903808 || trainable%: 0.23851079820371554
INFO:__main__:Train dataset downloaded:
 Dataset({
    features: ['input_ids'],
    num_rows: 274
})
INFO:__main__:Number of tokens for the training: 561152
  0%|                                                   | 0/822 [00:00<?, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/ext3/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The de

In [64]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
def generate(prompt: str, max_new_tokens: int, temperature: float, do_sample: bool) -> None:
        # Import the model
#         config = PeftConfig.from_pretrained(hf_repo)
        model = AutoModelForCausalLM.from_pretrained(config.model_name, return_dict=True, load_in_8bit=True)
        tokenizer = AutoTokenizer.from_pretrained(config.model_name)
        # Load the Lora model
        model = PeftModel.from_pretrained(model, "/scratch/sa6981/llm_unlearn/finetune_copyright/models/finetune_1.3b")

        # Generate text
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
#         inputs.cuda()
#         print(inputs)
#         print(inputs.shape)
        tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            top_p=0.95, top_k=50,
        )
        print(tokenizer.decode(tokens[0]))

In [4]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import config

In [66]:
prompt = "The hobbits were so suprised seeing their friend"
generate(prompt, 256, 0.3, True)

</s>The hobbits were so suprised seeing their friend again that they were all silent.</s>


In [54]:
prompt = "‘Get up, Sam!’ said"
generate(prompt, 128, 0.3, True)

</s>‘Get up, Sam!’ said Frodo. ‘You’re not going to be left behind.’ ‘I’m not going to be left behind!’ said Sam. ‘I’ll go with you!’ ‘No,’ said Frodo. ‘You can’t. You’re too weak.’ ‘I’m not weak,’ said Sam. ‘I’m not going to be left behind.’ ‘You’re not going to be left behind, Sam!’ said Frodo. ‘You


## generate for whole test dataset

In [2]:
import pickle
with open('dataset_dict.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [74]:
dataset['test']

Dataset({
    features: ['input_ids'],
    num_rows: 31
})

In [46]:
type(dataset['test'])

datasets.arrow_dataset.Dataset

In [70]:
def get_pred_from_peft_1_3b(dataset,max_new_tokens: int, temperature: float, do_sample: bool):
    model = AutoModelForCausalLM.from_pretrained(config.model_name, return_dict=True, load_in_8bit=True)
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    # Load the Lora model
    model = PeftModel.from_pretrained(model, "/scratch/sa6981/llm_unlearn/finetune_copyright/models/finetune_1.3b")
    tokenized_dataset = dataset.map(lambda example: example["input_ids"], batched=True)
    # Run the language model on the tokenized dataset
    input_ids = tokenized_dataset.to(device)
    print(input_ids)
#     print(input_ids.shape)
#     outputs = model.generate(input_ids)
    tokens = model.generate(
            input_ids = input_ids,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            top_p=0.95, top_k=50,
        )
    outputs = tokenizer.batch_decode(tokens, skip_special_tokens=True)
    
    return outputs
    

In [ ]:
dataset["test"]['input_ids']

In [ ]:
generated_seq = get_pred_from_peft_1_3b(dataset['test'],256, 0.3, True)

In [ ]:
from torch.utils.data import DataLoader

# Assuming dataset['test'] is already tokenized and has 'input_ids'
tokenized_dataset = dataset['test']

# Create a DataLoader for batching
loader = DataLoader(tokenized_dataset, batch_size=32, shuffle=False)

# List to store generated outputs
generated_outputs = []

model = AutoModelForCausalLM.from_pretrained(config.model_name, return_dict=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
# Load the Lora model
model = PeftModel.from_pretrained(model, "/scratch/sa6981/llm_unlearn/finetune_copyright/models/finetune_1.3b")
# Set the model to evaluation mode
model.eval()

# Iterate through the DataLoader
for batch in loader:
    # Assuming 'input_ids' is the key for the input_ids feature
    input_ids = batch['input_ids']# Move to the appropriate device (e.g., GPU)
    input_ids_tensor = torch.tensor(input_ids).unsqueeze(0)
    print(len(input_ids))
#     print(input_ids)
    # Generate outputs from the language model
    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids_tensor,
                                max_new_tokens=64,
            temperature=0.3,
            do_sample=True,
            top_p=0.95, top_k=50,)

    # Append the generated outputs to the list
    generated_outputs.extend(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))